In [20]:
# https://github.com/h2oai/sql-sidekick/releases
#!python3 -m pip install --force-reinstall sql_sidekick-0.2.2-py3-none-any.whl

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
from sidekick.prompter import db_setup, ask
from sidekick.schema_generator import generate_schema
from sidekick.utils import setup_dir, list_models

In [4]:
base_path = "./"
cache_path = f"{base_path}/var/lib/tmp"
setup_dir(base_path)

In [5]:
import pandas as pd
f = pd.read_csv("./sleep_health_and_lifestyle_dataset.csv")

In [6]:
# Env variables
import os

os.environ['OPENAI_API_KEY'] = ""
os.environ['H2O_BASE_MODEL_URL'] = 'http://38.128.233.247'
os.environ['H2O_BASE_MODEL_API_KEY'] = ""
# To get access to h2ogpte endpoint, reach out to cloud-feedback@h2o.ai
os.environ['RECOMMENDATION_MODEL_REMOTE_URL'] = "https://h2ogpte.genai.h2o.ai"  # e.g. https://<>.h2ogpte.h2o.ai
os.environ['RECOMMENDATION_MODEL_API_KEY'] = ""

In [7]:
HOST_NAME = "localhost"
USER_NAME = "sqlite"
PASSWORD = "abc"
DB_NAME = "querydb"
PORT = "5432"


# Given .csv file, auto-generate schema
# Download dataset --> https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset
data_path = "./sleep_health_and_lifestyle_dataset.csv"
table_name = "sleep_health_eda"

r, table_info_path = generate_schema(data_path=data_path, output_path=f"{cache_path}/{table_name}_table_info.jsonl")

llm_response, err = db_setup(
                db_name=DB_NAME,
                hostname=HOST_NAME,
                user_name=USER_NAME,
                password=PASSWORD,
                port=PORT,
                table_info_path=table_info_path,
                table_samples_path=data_path,
                table_name=table_name,
                local_base_path=base_path
            )

 Information supplied:
 querydb, localhost, sqlite, abc, 5432
Database already exists!
Table name: sleep_health_eda


2024-01-27 20:35:06.568 | DEBUG    | sidekick.db_config:_extract_schema_info:162 - Using schema information from: .//var/lib/tmp/sleep_health_eda_table_info.jsonl
2024-01-27 20:35:06.572 | DEBUG    | sidekick.db_config:create_table:186 - Schema info used for creating table:
 Person_ID NUMERIC,
Gender TEXT COLLATE NOCASE,
Age NUMERIC,
Occupation TEXT COLLATE NOCASE,
Sleep_Duration NUMERIC,
Quality_of_Sleep NUMERIC,
Physical_Activity_Level NUMERIC,
Stress_Level NUMERIC,
BMI_Category TEXT COLLATE NOCASE,
Blood_Pressure TEXT COLLATE NOCASE,
Heart_Rate NUMERIC,
Daily_Steps NUMERIC,
Sleep_Disorder TEXT COLLATE NOCASE
2024-01-27 20:35:06.578 | INFO     | sidekick.db_config:create_table:198 - Table created: sleep_health_eda


Checked table sleep_health_eda exists in the DB.


2024-01-27 20:35:06.586 | DEBUG    | sidekick.db_config:add_samples:222 - Adding sample values to table: ./sleep_health_and_lifestyle_dataset.csv
2024-01-27 20:35:06.597 | DEBUG    | sidekick.db_config:add_samples:228 - Inserting chunk: 0
2024-01-27 20:35:06.755 | INFO     | sidekick.db_config:add_samples:233 - Data inserted into table: sleep_health_eda
2024-01-27 20:35:06.759 | INFO     | sidekick.db_config:add_samples:238 - Number of rows inserted: 2618


Created a Database querydb. Inserted sample values from ./sleep_health_and_lifestyle_dataset.csv into table sleep_health_eda, please ask questions!


In [9]:
# List supported models
list_models()

['h2ogpt-sql-sqlcoder2-4bit',
 'h2ogpt-sql-sqlcoder-34b-alpha-4bit',
 'h2ogpt-sql-nsql-llama-2-7B-4bit',
 'h2ogpt-sql-sqlcoder2',
 'h2ogpt-sql-sqlcoder-34b-alpha',
 'h2ogpt-sql-nsql-llama-2-7B',
 'gpt-3.5-turbo',
 'gpt-4-8k',
 'gpt-4-1106-preview-128k']

In [10]:
# Use `list_models()` to check other supported models

def query(
    question: str,
    table_name,
    table_info_path: str,
    sample_qna_path: str,
    regenerate=False,
    regenerate_with_options=False,
):
    """Asks question and returns SQL."""
    base_path = "./"
    # self_correction is enabled by default, set to False if not needed.
    res = ask(
        question=question,
        table_info_path=table_info_path,
        sample_queries_path=sample_qna_path,
        table_name=table_name,
        is_command=False,
        model_name="h2ogpt-sql-sqlcoder2-4bit",  #Other default model option: h2ogpt-sql-sqlcoder-34b-alpha
        is_regenerate=regenerate,
        is_regen_with_options=regenerate_with_options,
        execute_query=False,
        local_base_path=base_path,
    )
    return res

In [11]:
res = query("What is the average sleep duration for each gender?", table_name="sleep_health_eda", 
            table_info_path=table_info_path, sample_qna_path=None)

2024-01-27 20:35:33.226 | INFO     | sidekick.prompter:ask:500 - Table in use: ['sleep_health_eda']
2024-01-27 20:35:33.229 | INFO     | sidekick.prompter:ask:501 - SQL dialect for generation: sqlite
2024-01-27 20:35:33.231 | INFO     | sidekick.prompter:ask:534 - Setting context...
2024-01-27 20:35:33.232 | INFO     | sidekick.prompter:ask:535 - Question: What is the average sleep duration for each gender?
2024-01-27 20:35:33.234 | DEBUG    | sidekick.prompter:ask:553 - Table info path: .//var/lib/tmp/sleep_health_eda_table_info.jsonl
2024-01-27 20:35:33.235 | INFO     | sidekick.utils:is_resource_low:355 - Number of GPUs: 1
2024-01-27 20:35:33.236 | DEBUG    | sidekick.utils:is_resource_low:359 - Information on device: 0
2024-01-27 20:35:35.049 | INFO     | sidekick.utils:is_resource_low:362 - Total Memory: 23GB
2024-01-27 20:35:35.055 | INFO     | sidekick.utils:is_resource_low:363 - Free GPU memory: 20GB
2024-01-27 20:35:35.057 | INFO     | sidekick.query:__new__:83 - Loading local

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/pramit/.jupyterven/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-27 20:36:22.408 | DEBUG    | sidekick.utils:semantic_search:160 - Probable context: ["if patterns like 'current time' or 'now' occurs in question", "if patterns like 'total number', or 'list' occurs in question", 'detailed summary', 'summary']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-27 20:36:22.444 | DEBUG    | sidekick.utils:semantic_search:168 - Similarity score for: if patterns like 'current time' or 'now' occurs in question: 0.8459207869447033
2024-01-27 20:36:22.446 | DEBUG    | sidekick.utils:semantic_search:168 - Similarity score for: if patterns like 'total number', or 'list' occurs in question: 0.8319947353454415
2024-01-27 20:36:22.448 | DEBUG    | sidekick.utils:semantic_search:168 - Similarity score for: detailed summary: 0.8346069603076574
2024-01-27 20:36:22.449 | DEBUG    | sidekick.utils:semantic_search:168 - Similarity score for: summary: 0.8394152180082535
2024-01-27 20:36:22.450 | DEBUG    | sidekick.utils:semantic_search:184 - Sorted context: []
2024-01-27 20:36:22.451 | DEBUG    | sidekick.query:generate_sql:486 - Filter Context: []
2024-01-27 20:36:22.452 | INFO     | sidekick.query:generate_sql:494 - Filtering Question/Query pairs ...
2024-01-27 20:36:22.453 | INFO     | sidekick.query:generate_sql:496 - Number of context queries fou

Exiting...


In [12]:
print(f"Question = {res[0][0]}")
print("----")
print(f"Generated SQL = {res[0][1]}")

Question = **Generated response for question,**
What is the average sleep duration for each gender?

----
Generated SQL = ``` sql
SELECT "gender",
       AVG("sleep_duration") AS "average_sleep_duration"
FROM "sleep_health_eda"
GROUP BY "gender"
LIMIT 100
```




In [16]:
# On using re-generation flag we toggle the temperature values between 0 and 1 alternating between low 
# (focus/conservative generation and high values (random/creative generation)
res = query("What are the most common occupations among individuals in the dataset?", table_name="sleep_health_eda", 
            table_info_path=table_info_path, sample_qna_path=None, regenerate=True)

2024-01-27 20:39:50.016 | INFO     | sidekick.prompter:ask:500 - Table in use: ['sleep_health_eda']
2024-01-27 20:39:50.017 | INFO     | sidekick.prompter:ask:501 - SQL dialect for generation: sqlite
2024-01-27 20:39:50.018 | INFO     | sidekick.prompter:ask:534 - Setting context...
2024-01-27 20:39:50.019 | INFO     | sidekick.prompter:ask:535 - Question: What are the most common occupations among individuals in the dataset?
2024-01-27 20:39:50.020 | DEBUG    | sidekick.prompter:ask:553 - Table info path: .//var/lib/tmp/sleep_health_eda_table_info.jsonl
2024-01-27 20:39:50.021 | INFO     | sidekick.utils:is_resource_low:355 - Number of GPUs: 1
2024-01-27 20:39:50.022 | DEBUG    | sidekick.utils:is_resource_low:359 - Information on device: 0
2024-01-27 20:39:50.023 | INFO     | sidekick.utils:is_resource_low:362 - Total Memory: 23GB
2024-01-27 20:39:50.024 | INFO     | sidekick.utils:is_resource_low:363 - Free GPU memory: 8GB
2024-01-27 20:39:50.038 | INFO     | sidekick.utils:_check_f

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-27 20:39:50.099 | DEBUG    | sidekick.utils:semantic_search:160 - Probable context: ["if patterns like 'current time' or 'now' occurs in question", "if patterns like 'total number', or 'list' occurs in question", 'detailed summary', 'summary']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-27 20:39:50.139 | DEBUG    | sidekick.utils:semantic_search:168 - Similarity score for: if patterns like 'current time' or 'now' occurs in question: 0.8284876985286928
2024-01-27 20:39:50.141 | DEBUG    | sidekick.utils:semantic_search:168 - Similarity score for: if patterns like 'total number', or 'list' occurs in question: 0.8591431101102107
2024-01-27 20:39:50.143 | DEBUG    | sidekick.utils:semantic_search:168 - Similarity score for: detailed summary: 0.8650206131706182
2024-01-27 20:39:50.146 | DEBUG    | sidekick.utils:semantic_search:168 - Similarity score for: summary: 0.8724867083448907
2024-01-27 20:39:50.147 | DEBUG    | sidekick.utils:semantic_search:184 - Sorted context: []
2024-01-27 20:39:50.148 | DEBUG    | sidekick.query:generate_sql:486 - Filter Context: []
2024-01-27 20:39:50.149 | INFO     | sidekick.query:generate_sql:494 - Filtering Question/Query pairs ...
2024-01-27 20:39:50.150 | INFO     | sidekick.query:generate_sql:496 - Number of context queries fou

Exiting...


In [17]:
print(f"Question = {res[0][0]}")
print("----")
print(f"Generated SQL = {res[0][1]}")

Question = **Generated response for question,**
What are the most common occupations among individuals in the dataset?

----
Generated SQL = ``` sql
SELECT "occupation",
       COUNT(1) AS "COUNT"
FROM "sleep_health_eda"
GROUP BY "occupation"
ORDER BY "COUNT" DESC
LIMIT 100
```




In [18]:
# Alternate options
res = query("What is the average sleep duration for each gender?", table_name="sleep_health_eda", 
            table_info_path=table_info_path, sample_qna_path=None, regenerate_with_options=True)

2024-01-27 20:39:56.595 | INFO     | sidekick.prompter:ask:500 - Table in use: ['sleep_health_eda']
2024-01-27 20:39:56.597 | INFO     | sidekick.prompter:ask:501 - SQL dialect for generation: sqlite
2024-01-27 20:39:56.598 | INFO     | sidekick.prompter:ask:534 - Setting context...
2024-01-27 20:39:56.599 | INFO     | sidekick.prompter:ask:535 - Question: What is the average sleep duration for each gender?
2024-01-27 20:39:56.601 | DEBUG    | sidekick.prompter:ask:553 - Table info path: .//var/lib/tmp/sleep_health_eda_table_info.jsonl
2024-01-27 20:39:56.602 | INFO     | sidekick.utils:is_resource_low:355 - Number of GPUs: 1
2024-01-27 20:39:56.604 | DEBUG    | sidekick.utils:is_resource_low:359 - Information on device: 0
2024-01-27 20:39:56.605 | INFO     | sidekick.utils:is_resource_low:362 - Total Memory: 23GB
2024-01-27 20:39:56.607 | INFO     | sidekick.utils:is_resource_low:363 - Free GPU memory: 8GB
2024-01-27 20:39:56.629 | INFO     | sidekick.utils:_check_file_info:469 - Usin

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-27 20:39:56.699 | DEBUG    | sidekick.utils:semantic_search:160 - Probable context: ["if patterns like 'current time' or 'now' occurs in question", "if patterns like 'total number', or 'list' occurs in question", 'detailed summary', 'summary']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-27 20:39:56.752 | DEBUG    | sidekick.utils:semantic_search:168 - Similarity score for: if patterns like 'current time' or 'now' occurs in question: 0.8459207869447033
2024-01-27 20:39:56.755 | DEBUG    | sidekick.utils:semantic_search:168 - Similarity score for: if patterns like 'total number', or 'list' occurs in question: 0.8319947353454415
2024-01-27 20:39:56.759 | DEBUG    | sidekick.utils:semantic_search:168 - Similarity score for: detailed summary: 0.8346069603076574
2024-01-27 20:39:56.763 | DEBUG    | sidekick.utils:semantic_search:168 - Similarity score for: summary: 0.8394152180082535
2024-01-27 20:39:56.764 | DEBUG    | sidekick.utils:semantic_search:184 - Sorted context: []
2024-01-27 20:39:56.766 | DEBUG    | sidekick.query:generate_sql:486 - Filter Context: []
2024-01-27 20:39:56.767 | INFO     | sidekick.query:generate_sql:494 - Filtering Question/Query pairs ...
2024-01-27 20:39:56.769 | INFO     | sidekick.query:generate_sql:496 - Number of context queries fou

Exiting...


In [19]:
print(f"Question = {res[0][0]}")
print("----Options----")
for _r in res[1]:
    print(_r)

Question = **Generated response for question,**
What is the average sleep duration for each gender?

----Options----
Option 1: (_probability_: 0.381034255027771)
``` sql
SELECT gender,
       AVG(sleep_duration) AS average_sleep_duration
FROM sleep_health_eda
GROUP BY gender
ORDER BY average_sleep_duration DESC NULLS LAST
LIMIT 100;
```



Option 2: (_probability_: 0.2624567449092865)
``` sql
SELECT AVG(sleep_duration) AS average_sleep_duration,
       gender
FROM sleep_health_eda
GROUP BY gender
LIMIT 100;
```



Option 3: (_probability_: 0.22498156130313873)
``` sql
SELECT Gender,
       AVG(Sleep_Duration) AS average_duration
FROM sleep_health_eda
GROUP BY Gender
ORDER BY average_duration DESC NULLS LAST
LIMIT 100;
```



Option 4: (_probability_: 0.13085876405239105)
``` sql
SELECT 'Gender',
       AVG('Sleep_Duration') AS average_sleep_duration
FROM'sleep_health_eda'
GROUP BY 'Gender'
ORDER BY average_sleep_duration DESC NULLS LAST
LIMIT 100;
```



Option 5: (_probability_: 0.000